# Tutorial 5: Strong Normalization

*Every Typed Term Terminates*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-typed-passages/blob/main/notebooks/05-strong-normalization.ipynb)

---

## A Note from the Archives

> *Year 830, Capital Archives*
>
> Brennis Mund had done it. After thirty years of work, he had proven the theorem that would ensure the Archive's safety forever.
>
> **Strong Normalization**: Every well-typed term terminates.
>
> No more seven-hour halts. No more Omega variants slipping through review. If a passage passed the typing discipline, it was guaranteed to reduce to a normal form.
>
> His father Kelleth had seen the problem. Brennis had found the solution.

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. State the Strong Normalization theorem
2. Distinguish between weak and strong normalization
3. Understand why types guarantee termination
4. Recognize that this solves the Omega problem

## Setup

In [ ]:
import pandas as pd

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

expressions_df = pd.read_csv(BASE_URL + "typed_passage_expressions.csv")
traces_df = pd.read_csv(BASE_URL + "normalization_traces.csv")

print(f"Loaded {len(expressions_df)} expressions")
print(f"Loaded {len(traces_df)} reduction steps")

## 1. What is Normalization?

An expression **normalizes** if it eventually reduces to a normal form (a value that cannot reduce further).

**Weak normalization**: There *exists* a reduction sequence that terminates

**Strong normalization**: *Every* reduction sequence terminates

The difference matters because lambda calculus allows choice of which redex to reduce.

In [ ]:
# Weak vs Strong normalization
print("Normalization Concepts:")
print()
print("  WEAK: At least one path to normal form")
print("  Example: (λx.y) Ω")
print("    - Reduce (λx.y) Ω → y  ✓ (ignoring Ω)")
print("    - Reduce Ω first → loops forever ✗")
print("  Weakly normalizing (one good path exists)")
print()
print("  STRONG: ALL paths lead to normal form")
print("  Example: (λx.x)(λy.y)")
print("    - Only one redex, reduces to λy.y ✓")
print("  Strongly normalizing (no bad paths exist)")

## 2. The Strong Normalization Theorem

**Theorem (Strong Normalization for STLC)**:

> If $\Gamma \vdash M : \tau$, then every reduction sequence starting from $M$ is finite.

In words: **Every well-typed term terminates, no matter how you reduce it.**

This is the ultimate answer to the Omega problem.

In [ ]:
# All typed expressions terminate
typed_exprs = expressions_df[expressions_df['typable'] == True]

print("Typed expressions and termination:")
print()

terminates_count = typed_exprs['terminates'].sum()
total_typed = len(typed_exprs)

print(f"  Typed expressions: {total_typed}")
print(f"  Terminate: {terminates_count}")
print(f"  Percentage: {100*terminates_count/total_typed:.0f}%")
print()
print("  Strong Normalization guarantees 100% termination!")

## 3. Why Types Guarantee Termination

The key insight: every reduction **decreases** some measure.

For simply typed lambda calculus, we can use the **size of the type** as a measure:

- Base types have size 1
- Arrow types: $|\tau_1 \to \tau_2| = |\tau_1| + |\tau_2| + 1$

When we reduce $(\lambda x.M)\ N$, the substitution $M[N/x]$ has a "smaller" type structure (in a precise sense).

In [ ]:
# Intuition for why types help
print("Why Types Guarantee Termination (Intuition):")
print()
print("  The type of a redex (λx:τ₁.M):τ₁→τ₂ applied to N:τ₁")
print("  is τ₂.")
print()
print("  After reduction, M[N/x] has type τ₂.")
print()
print("  The 'complexity' of the typing derivation decreases.")
print("  Since it can't decrease forever, reduction terminates.")
print()
print("  (The formal proof uses 'reducibility candidates'.)")

## 4. Observing Termination in Traces

In [ ]:
# Show reduction lengths for various expressions
print("Reduction lengths for typed expressions:")
print()

for trace_id in traces_df['trace_id'].unique():
    trace = traces_df[traces_df['trace_id'] == trace_id]
    expr = trace.iloc[0]['expression']
    type_ = trace.iloc[0]['type']
    steps = len(trace) - 1  # Subtract 1 because we count transitions
    final = trace[trace['is_value'] == True]
    
    if len(final) > 0:
        final_term = final.iloc[0]['current_term']
        if len(final_term) > 20:
            final_term = final_term[:17] + "..."
        print(f"  {trace_id}: {steps} steps → {final_term}")

## 5. The SUCC Chain Example

In [ ]:
# Trace SUCC ZERO
trace_succ = traces_df[traces_df['trace_id'] == 'NT-004']

print("SUCC ZERO reduction:")
print(f"Type: {trace_succ.iloc[0]['type']}")
print()

for _, row in trace_succ.iterrows():
    term = row['current_term']
    if len(term) > 50:
        term = term[:47] + "..."
    marker = " [VALUE]" if row['is_value'] else ""
    print(f"  Step {row['step_number']}: {term}{marker}")

print()
print("  SUCC ZERO = ONE. Terminates in finite steps!")

## 6. Contrast: Omega Does NOT Normalize

Omega reduces to itself forever:

$$\Omega = (\lambda x.x\ x)(\lambda x.x\ x) \to \Omega \to \Omega \to \ldots$$

But Omega is **untypable**! Strong Normalization only applies to well-typed terms.

In [ ]:
# Show Omega doesn't normalize
omega = expressions_df[expressions_df['expression'].str.contains('\(λx\.x x\)\(λx\.x x\)', na=False, regex=True)]

if omega.empty:
    # Try simpler pattern
    omega = expressions_df[expressions_df['expression_id'] == 'TPE-009']

print("Omega and termination:")
print()
if not omega.empty:
    row = omega.iloc[0]
    print(f"  Expression: {row['expression']}")
    print(f"  Typable: {row['typable']}")
    print(f"  Terminates: {row['terminates']}")
    print(f"  Reduction steps: {row['reduction_steps']}")
print()
print("  Omega is untypable, so Strong Normalization doesn't apply.")
print("  This is exactly the point — types exclude non-terminating terms!")

## 7. The Price of Termination

Strong Normalization comes at a cost: we lose the ability to express **general recursion**.

The Y combinator — which enables recursion — is untypable:

$$Y = \lambda f.(\lambda x.f(x\ x))(\lambda x.f(x\ x))$$

This contains $(x\ x)$, which requires $\tau = \tau \to \sigma$ — no finite type works.

In [ ]:
# Y combinator is untypable
y_comb = expressions_df[expressions_df['expression_id'] == 'TPE-011']

print("The Y combinator:")
print()
if not y_comb.empty:
    row = y_comb.iloc[0]
    print(f"  Expression: {row['expression']}")
    print(f"  Typable: {row['typable']}")
    print(f"  Terminates: {row['terminates']}")
print()
print("  Y enables recursion but is untypable.")
print("  This is the tradeoff: termination vs full recursion.")

## 8. What Can We Still Compute?

Despite losing Y, the simply typed lambda calculus can still compute:

- All primitive recursive functions (via Church numerals)
- Many useful data transformations
- Anything that doesn't require unbounded iteration

Brennis Mund later proposed **primitive recursion** as a typed alternative to Y.

In [ ]:
# What's still computable?
print("What STLC can still compute:")
print()
print("  ✓ Addition, multiplication of Church numerals")
print("  ✓ Boolean logic (AND, OR, NOT)")
print("  ✓ List operations (map, fold with bounded length)")
print("  ✓ Function composition")
print("  ✓ Anything with bounded iteration")
print()
print("What STLC cannot compute:")
print()
print("  ✗ General recursion (factorial, Fibonacci)")
print("  ✗ While loops")
print("  ✗ Turing-complete computation")
print()
print("  But with primitive recursion, we get factorial back!")

## Summary

In this tutorial, we learned:

1. **Strong Normalization**: Every well-typed term terminates (all reduction paths are finite)
2. This is stronger than **weak normalization** (just one path terminates)
3. **Types guarantee termination** because each reduction step decreases some measure
4. **Omega is excluded** because it's untypable — this is the whole point!
5. **The tradeoff**: We lose general recursion (Y is untypable)
6. **Primitive recursion** can recover many useful recursive patterns

In the next tutorial, we'll explore this tradeoff between safety and expressiveness.

---

## Exercises

1. Is $(\lambda x.y)\ \Omega$ strongly normalizing, weakly normalizing, or non-normalizing? (In untyped calculus)

2. Why can't we type $\lambda x.x\ x\ x\ x\ x$? (Hint: what equation would $\tau$ satisfy?)

3. How many reduction steps does $((\lambda x.\lambda y.x)\ a)\ b$ take to reach normal form?

4. If we added a "loop" primitive to STLC that reduced to itself, would Strong Normalization still hold?

---

*Next: Tutorial 6 - The Expressiveness Tradeoff*